In [419]:
!pip install haversine
!pip3 install --upgrade setuptools
!pip3 install shapely

     |████████████████████████████████| 1.1 MB 5.2 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [420]:
import pandas as pd
import sys
import math
from scipy.spatial import distance
from functools import reduce
import time
import requests
import logging
import requests
from haversine import haversine
from shapely.geometry import Point, Polygon

In [439]:
address = pd.DataFrame({'분류':['병원', '편의점', '학교', '편의점', '병원', '카페', '카페', '카페', '문화센터'], 
                        '도로명주소':['서울특별시 용산구 효창동 5-99', '서울특별시 용산구 갈월동 16', '서울특별시 용산구 청파로47길 100', '서울특별시 용산구 효창동 3-292', '서울특별시 용산구 갈월동 69-3','서울특별시 용산구 갈월동 69-3','서울특별시 용산구 갈월동 69-3','서울특별시 용산구 갈월동 69-3','서울특별시 용산구 갈월동 69-3'],
                        '상세주소':['연세세림내과', 'CU숙대입구역점', '숙명여자대학교 제1캠퍼스', 'GS25 효창점', '하나이비인후과', '스타벅스', '엔젤리너스', '투썸플레이스', '용산구문화센터'],
                        '경도':[126.96207723819813, 126.97216672677722, 126.96463,126.96323671546, 126.97194477926477, 126.9926368740932, 126.99730216821392,126.99506808374375, 127.0030023688794], 
                        '위도':[37.54045738249753,37.5455250030874,37.54665, 37.54692932830024, 37.54522978032181, 37.539638863594405,37.538062753471344, 37.53424608426737, 37.53226602876014]})
address

,분류,도로명주소,상세주소,경도,위도
0,병원,서울특별시 용산구 효창동 5-99,연세세림내과,126.962077,37.540457
1,편의점,서울특별시 용산구 갈월동 16,CU숙대입구역점,126.972167,37.545525
2,학교,서울특별시 용산구 청파로47길 100,숙명여자대학교 제1캠퍼스,126.964630,37.546650
3,편의점,서울특별시 용산구 효창동 3-292,GS25 효창점,126.963237,37.546929
4,병원,서울특별시 용산구 갈월동 69-3,하나이비인후과,126.971945,37.545230
5,카페,서울특별시 용산구 갈월동 69-3,스타벅스,126.992637,37.539639
6,카페,서울특별시 용산구 갈월동 69-3,엔젤리너스,126.997302,37.538063
7,카페,서울특별시 용산구 갈월동 69-3,투썸플레이스,126.995068,37.534246
8,문화센터,서울특별시 용산구 갈월동 69-3,용산구문화센터,127.003002,37.532266


In [440]:
add_df = pd.DataFrame({'분류':[], 
                        '도로명주소':[],
                        '상세주소':[],
                        '경도':[], 
                        '위도':[],
                        '희망시간':[]})
data_num = int(input("입력할 시설의 개수를 쓰시오: "))
data_input =[]
data_time = []
for i in range(data_num):
  data_input.append(input("시설을 입력해주세요: "))
  data_time.append(input("희망 소요시간(분단위)을 입력해주세요: "))
data_time = list(map(int, data_time))
for j, i in zip(data_input, data_time):
  address_ab = address.loc[address['분류'] == j]
  add_df = add_df.append(address_ab, ignore_index=True)
  add_df.loc[add_df["분류"] == j, "희망시간"] = int(i)

add_df

입력할 시설의 개수를 쓰시오: 5
시설을 입력해주세요: 편의점
희망 소요시간(분단위)을 입력해주세요: 1
시설을 입력해주세요: 카페
희망 소요시간(분단위)을 입력해주세요: 5
시설을 입력해주세요: 병원
희망 소요시간(분단위)을 입력해주세요: 1
시설을 입력해주세요: 학교
희망 소요시간(분단위)을 입력해주세요: 15
시설을 입력해주세요: 문화센터
희망 소요시간(분단위)을 입력해주세요: 20


,분류,도로명주소,상세주소,경도,위도,희망시간
0,편의점,서울특별시 용산구 갈월동 16,CU숙대입구역점,126.972167,37.545525,1.0
1,편의점,서울특별시 용산구 효창동 3-292,GS25 효창점,126.963237,37.546929,1.0
2,카페,서울특별시 용산구 갈월동 69-3,스타벅스,126.992637,37.539639,5.0
3,카페,서울특별시 용산구 갈월동 69-3,엔젤리너스,126.997302,37.538063,5.0
4,카페,서울특별시 용산구 갈월동 69-3,투썸플레이스,126.995068,37.534246,5.0
5,병원,서울특별시 용산구 효창동 5-99,연세세림내과,126.962077,37.540457,1.0
6,병원,서울특별시 용산구 갈월동 69-3,하나이비인후과,126.971945,37.545230,1.0
7,학교,서울특별시 용산구 청파로47길 100,숙명여자대학교 제1캠퍼스,126.964630,37.546650,15.0
8,문화센터,서울특별시 용산구 갈월동 69-3,용산구문화센터,127.003002,37.532266,20.0


In [441]:
add_count = add_df['분류'].value_counts()
min_count = add_count.axes[0][0]
count = add_count[0]
for i in range(len(add_count)):
  if add_count[i]<count:
    min_count = add_count.axes[0][i]

min_count

'문화센터'

In [442]:
a = address.loc[address['분류'] == min_count, ["분류",'위도', '경도']]
a = (a['위도'].values[0], a['경도'].values[0])
s = []
for y, x in zip(add_df['위도'], add_df['경도']):
  b = (y, x)
  s.append(distance.euclidean(a, b))
add_df['거리'] = s

add_df

,분류,도로명주소,상세주소,경도,위도,희망시간,거리
0,편의점,서울특별시 용산구 갈월동 16,CU숙대입구역점,126.972167,37.545525,1.0,0.033565
1,편의점,서울특별시 용산구 효창동 3-292,GS25 효창점,126.963237,37.546929,1.0,0.042383
2,카페,서울특별시 용산구 갈월동 69-3,스타벅스,126.992637,37.539639,5.0,0.012720
3,카페,서울특별시 용산구 갈월동 69-3,엔젤리너스,126.997302,37.538063,5.0,0.008130
4,카페,서울특별시 용산구 갈월동 69-3,투썸플레이스,126.995068,37.534246,5.0,0.008178
5,병원,서울특별시 용산구 효창동 5-99,연세세림내과,126.962077,37.540457,1.0,0.041737
6,병원,서울특별시 용산구 갈월동 69-3,하나이비인후과,126.971945,37.545230,1.0,0.033655
7,학교,서울특별시 용산구 청파로47길 100,숙명여자대학교 제1캠퍼스,126.964630,37.546650,15.0,0.040980
8,문화센터,서울특별시 용산구 갈월동 69-3,용산구문화센터,127.003002,37.532266,20.0,0.000000


In [443]:
address_final = add_df.loc[add_df.groupby(['분류'])['거리'].idxmin()]
address_final = address_final.sort_values('거리')
address_final = address_final.reset_index(drop = True)

In [444]:
address_final

,분류,도로명주소,상세주소,경도,위도,희망시간,거리
0,문화센터,서울특별시 용산구 갈월동 69-3,용산구문화센터,127.003002,37.532266,20.0,0.000000
1,카페,서울특별시 용산구 갈월동 69-3,엔젤리너스,126.997302,37.538063,5.0,0.008130
2,편의점,서울특별시 용산구 갈월동 16,CU숙대입구역점,126.972167,37.545525,1.0,0.033565
3,병원,서울특별시 용산구 갈월동 69-3,하나이비인후과,126.971945,37.545230,1.0,0.033655
4,학교,서울특별시 용산구 청파로47길 100,숙명여자대학교 제1캠퍼스,126.964630,37.546650,15.0,0.040980


In [445]:
#희망시간 리스트로 변환하기
data_time = address_final["희망시간"].tolist()

In [459]:
address_fx = address_final["경도"].values.tolist()
address_fy = address_final["위도"].values.tolist()
xy = []
point = []

for i in range(len(address_fx)):
  xy.append(address_fx[i])
  xy.append(address_fy[i])
  point.append(xy)
  xy = []
e_point=point
point


[[127.0030023688794, 37.53226602876014],
 [126.99730216821392, 37.538062753471344],
 [126.97216672677722, 37.5455250030874],
 [126.97194477926477, 37.54522978032181],
 [126.96463, 37.54665]]

In [447]:
#최외각 노드(시설) 선정하기 
def convex_hull_graham(points):
    '''
    Returns points on convex hull in CCW order according to Graham's scan algorithm. 
    By Tom Switzer <thomas.switzer@gmail.com>.
    '''
    TURN_LEFT, TURN_RIGHT, TURN_NONE = (1, -1, 0)

    def cmp(a, b):
        return (a > b) - (a < b)

    def turn(p, q, r):
        return cmp((q[0] - p[0])*(r[1] - p[1]) - (r[0] - p[0])*(q[1] - p[1]), 0)

    def _keep_left(hull, r):
        while len(hull) > 1 and turn(hull[-2], hull[-1], r) != TURN_LEFT:
            hull.pop()
        if not len(hull) or hull[-1] != r:
            hull.append(r)
        return hull

    points = sorted(points)
    l = reduce(_keep_left, points, [])
    u = reduce(_keep_left, reversed(points), [])
    return l.extend(u[i] for i in range(1, len(u) - 1)) or l

In [448]:
point = convex_hull_graham(point)
point

[[126.96463, 37.54665],
 [127.0030023688794, 37.53226602876014],
 [126.99730216821392, 37.538062753471344],
 [126.97216672677722, 37.5455250030874]]

In [449]:
x = 0
y = 0
for i in point:
  x+=i[0]
  y+=i[1]
x = float(x/len(point))
y = float(y/len(point))
early = [x, y]
print(early)

[126.98427531596764, 37.54062594632972]


In [450]:
#소요시간 구하는 함수
def distance_time(start, end):
  logging.basicConfig(format='[%(levelname)s] %(asctime)s - %(message)s', level=logging.INFO, datefmt="%H:%M:%S")

  version = 1
  callback = "result"
  apiUrl = f"https://apis.openapi.sk.com/tmap/routes?version={version}&callback={callback}"

  headers = {
      "appKey": "l7xx0b316dc5f608415b8107bb2aa9fc7dbd",
      "Accept": "application/json",
      "Content-Type": "application/json; charset=UTF-8"
  }

  json = {
      "startX" : str(start[0]),
      "startY" : str(start[1]),

      "endX" : str(end[0]),
      "endY" : str(end[1]),
      "reqCoordType" : "WGS84GEO",
      "resCoordType" : "EPSG3857"
      # "searchOption" : searchOption,
      # "trafficInfo" : trafficInfochk
  }

  req = requests.post(apiUrl, headers= headers, json=json)
  j = req.json()

  #for feat in j["features"]:
      #logging.info(feat)

  feat = j["features"][0]
  dist = feat["properties"]["totalDistance"]
  time = feat["properties"]["totalTime"]
  logging.info(f"totalDistance = {dist}, totalTime = {time}")
  return(float(time/60))

In [451]:
#소요시간 리스트 함수
def time_point(point,early):
  time_list = []
  for i in point:
    time_list.append(distance_time(early, i))
  return time_list

times = time_point(point, early)
times

[INFO] 19:47:12 - totalDistance = 4274, totalTime = 678
[INFO] 19:47:13 - totalDistance = 2828, totalTime = 565
[INFO] 19:47:14 - totalDistance = 2454, totalTime = 508
[INFO] 19:47:14 - totalDistance = 3140, totalTime = 516


[11.3, 9.416666666666666, 8.466666666666667, 8.6]

In [452]:
def cal(times):
  result = 0 
  for val in times: 
    result += val
  time_ave = float(result/len(times))
  time_trans = []
  for i in times:
    time_trans.append(float(i/time_ave))
  return time_trans

In [453]:
time_average = cal(times)
time_average

[1.196294662549625, 0.996912218791354, 0.8963387737097485, 0.910454344949272]

In [454]:
print(point)

[[126.96463, 37.54665], [127.0030023688794, 37.53226602876014], [126.99730216821392, 37.538062753471344], [126.97216672677722, 37.5455250030874]]


In [455]:
point_poly = Polygon(point)

In [456]:
def time_weight(point, mid_points):
  new_point = []
  mid_a_point = mid_points
  #중간값과 기존 노드 사이의 소요시간 구하기
  time_points = time_point(point, mid_points)
  #시간 가중치 구하기
  time_weights = cal(time_points)

  #노드 수만큼 이동상수 리스트 만들기
  point_time = [0 for _ in range(len(point))]

  #이동상수 계산
  for i,j in zip(point, time_weights):
    new_point.append(i[0]*j)
    new_point.append(i[1]*j)
    point_time[point.index(i)] = new_point
    new_point=[]
  
  #(이동상수*노드)/(노드개수) 계산
  distance_x=0
  distance_y=0
  for i in point_time:
    distance_x+=i[0]
    distance_y+=i[1]
  distance_x/=len(point)
  distance_y/=len(point)


  # 중간값 걩신
  mid_points = [distance_x, distance_y]
  print("갱신된 중간값: ", end="")
  print(mid_points)
  test_code = Point(mid_points)

  #소요시간과 희망시간 조건 맞는지 확인 + 이동거리가 100미터 이하인지 확인 + 결정한 다각형에 중간값이 들어가는지 확인
  for x, y in zip(time_point(point, mid_points), data_time):
    if x < y or haversine((distance_y,distance_x),(mid_a_point[1],mid_a_point[0]), unit='m')<=100 or (test_code.within(point_poly)):
      result = False
    else:
      result = True
  #만약 조건이 맞으면 stop
  if result: 
    time_weight(point, mid_points)  
  else:
    return mid_points
    

In [457]:
print("초기 중간값: ", end="")
print(early)
mid=time_weight(point, early)


초기 중간값: [126.98427531596764, 37.54062594632972]


[INFO] 19:47:31 - totalDistance = 4274, totalTime = 680
[INFO] 19:47:32 - totalDistance = 2828, totalTime = 565
[INFO] 19:47:33 - totalDistance = 2454, totalTime = 508
[INFO] 19:47:34 - totalDistance = 3140, totalTime = 516


갱신된 중간값: [126.98323026505801, 37.54088477574151]


[INFO] 19:47:35 - totalDistance = 4268, totalTime = 679
[INFO] 19:47:35 - totalDistance = 2822, totalTime = 564
[INFO] 19:47:36 - totalDistance = 2448, totalTime = 507
[INFO] 19:47:37 - totalDistance = 3134, totalTime = 515


In [460]:
import folium
#초기 로딩 위치 = 용산구
map_osm = folium.Map(location = [37.547019088096604, 126.96451990527733], zoom_start=15)

for item in range(len(e_point)):
  folium.Marker([point[item][1],point[item][0]], popup="희망시설").add_to(map_osm)

folium.Marker([early[1],early[0]], popup="중간주거지점", icon=folium.Icon('green', icon='star')).add_to(map_osm)
folium.Marker([mid[1],mid[0]], popup="중간주거지점", icon=folium.Icon('red', icon='star')).add_to(map_osm)

map_osm